In [1]:
import pandas as pd
import requests
import numpy as np
import time
import folium

In [2]:
df = pd.read_csv('../data/data_gps.csv')

In [3]:
df.columns

Index(['NO', 'INVOICE NO', 'CUSTOMER NAME', 'ADDRESS', 'Invoice Value', 'Rep',
       'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [4]:
if 'ADDRESS' in df.columns and 'LATITUDE' in df.columns and 'LONGITUDE' in df.columns:
    unique_locations_df = df.drop_duplicates(subset=["ADDRESS"])[
        ["ADDRESS", "LATITUDE", "LONGITUDE"]
    ]
    
else:
    print("Address, Latitude, Longitude columns not found in the data.")
    unique_locations_df = None

In [5]:
unique_locations_df.shape

(28, 3)

In [6]:
len(unique_locations_df)

28

In [7]:
kiribathkumbura = (7.2726897, 80.5618714)
kiribathkumbura_data = {
    "ADDRESS": "Kiribathkumbura",
    "LATITUDE": kiribathkumbura[0],
    "LONGITUDE": kiribathkumbura[1],
}

In [8]:
unique_locations_df = pd.concat(
    [unique_locations_df, pd.DataFrame([kiribathkumbura_data])], ignore_index=True
)
unique_locations_df.shape

(29, 3)

In [9]:
def get_osrm_data(origin, destination):
    """
    Get the distance between two coordinates using OSRM API.
    :param origin: (latitude, longitude)
    :param destination: (latitude, longitude)
    :return: Distance in meters
    """
    osrm_base_url = "http://router.project-osrm.org/route/v1/car"
    url = f"{osrm_base_url}/{origin[1]},{origin[0]};{destination[1]},{destination[0]}?overview=full&geometries=geojson"
    responce = requests.get(url)

    if responce.status_code == 200:
        data = responce.json()
        if "routes" in data and len(data["routes"]) > 0:
            path_cords = data["routes"][0]["geometry"]["coordinates"]
            distance = data["routes"][0]["distance"] / 1000
            return path_cords, distance
        
    return None, np.inf

In [10]:
location_names = unique_locations_df['ADDRESS'].to_list()
location_cords = list(zip(unique_locations_df["LATITUDE"], unique_locations_df["LONGITUDE"]))

In [11]:
origin = location_cords[0]
destination = location_cords[1]
get_osrm_data(origin, destination)

([[80.210328, 8.263934],
  [80.210446, 8.26367],
  [80.210835, 8.26385],
  [80.212091, 8.264438],
  [80.215307, 8.265912],
  [80.216566, 8.266473],
  [80.221407, 8.268629],
  [80.221558, 8.268697],
  [80.223288, 8.269467],
  [80.223468, 8.269547],
  [80.224286, 8.269912],
  [80.226199, 8.270763],
  [80.227337, 8.27127],
  [80.227729, 8.271445],
  [80.229255, 8.272022],
  [80.229379, 8.272069],
  [80.230915, 8.27265],
  [80.231415, 8.272839],
  [80.231651, 8.272929],
  [80.233865, 8.273768],
  [80.235082, 8.27423],
  [80.235266, 8.274296],
  [80.238244, 8.275423],
  [80.240149, 8.276143],
  [80.242369, 8.277001],
  [80.245102, 8.278057],
  [80.246139, 8.278457],
  [80.248149, 8.279233],
  [80.248899, 8.2795],
  [80.250437, 8.280046],
  [80.250741, 8.280116],
  [80.252186, 8.280394],
  [80.252697, 8.280489],
  [80.255284, 8.281525],
  [80.256851, 8.282153],
  [80.258338, 8.282718],
  [80.258462, 8.282765],
  [80.260158, 8.282791],
  [80.262399, 8.283674],
  [80.264157, 8.28434],
  [80.26

In [12]:
loc_count = len(location_cords)
loc_count

29

In [13]:
distance_matrix = np.zeros((loc_count, loc_count))
route_data = []

In [14]:
for i in range(loc_count):
    for j in range(i+1, loc_count):
        origin = location_cords[i]
        destination = location_cords[j]

        path_cords, distance = get_osrm_data(origin, destination)

        distance_matrix[i][j] = distance
        distance_matrix[j][i] = distance

        route_data.append(
            {
                "origin": location_names[i],
                "destination": location_names[j],
                "distance": distance,
                "path_cords": path_cords,
            }
        )
        
        time.sleep(0.2)

In [16]:
import json

# Prepare the data for JSON storage
distance_matrix_data = {
    "locations": location_names,  # Store location names
    "cords": location_cords,
    "route_data": route_data,  # Store route data as a list of dictionaries
    "distance_matrix": distance_matrix.tolist(),  # Convert NumPy array to list for JSON
}

# Define JSON file path
json_file_path = "../data/osrm_distance_matrix.json"

# Save to JSON file
with open(json_file_path, "w") as json_file:
    json.dump(distance_matrix_data, json_file, indent=4)

# Display the JSON file path
json_file_path

'../data/osrm_distance_matrix.json'